In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.0.0


In [3]:
landed = pd.read_csv("data\LandedArea.csv")
apartments = pd.read_csv('data\ApartmentsandCondos.csv')

In [4]:
landed.head()

,S/N,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale
0,1,ONE TREE HILL COLLECTION,JALAN KELAWAR,Detached,10,CCR,Freehold,New Sale,1.0,15500000.0,-,5145.0,Land,-,3013.0,Feb-20
1,2,BAN GUAN PARK,HOLLAND ROAD,Detached,10,CCR,Freehold,Resale,1.0,15500000.0,-,9050.0,Land,-,1713.0,Feb-20
2,3,LANDED HOUSING DEVELOPMENT,CORONATION ROAD WEST,Detached,10,CCR,Freehold,Resale,1.0,9380000.0,-,6242.0,Land,-,1503.0,Feb-20
3,4,LANDED HOUSING DEVELOPMENT,JALAN AMPANG,Semi-detached,10,CCR,Freehold,Resale,1.0,6600000.0,-,4529.0,Land,-,1457.0,Feb-20
4,5,LANDED HOUSING DEVELOPMENT,JALAN JAMBU AYER,Detached,21,RCR,Freehold,Resale,1.0,8600000.0,-,6218.0,Land,-,1383.0,Feb-20


In [5]:
apartments.head()

,S/N,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqft),Type of Area,Floor Level,Unit Price ($psf),Date of Sale
0,1,STIRLING RESIDENCES,STIRLING ROAD,Apartment,3,RCR,99 yrs lease commencing from 2017,New Sale,1.0,2000000.0,-,1055.0,Strata,36 to 40,1896.0,Feb-20
1,2,PARC CLEMATIS,JALAN LEMPENG,Apartment,5,OCR,99 yrs lease commencing from 2019,New Sale,1.0,1624000.0,-,1044.0,Strata,01 to 05,1555.0,Feb-20
2,3,PARC CLEMATIS,JALAN LEMPENG,Apartment,5,OCR,99 yrs lease commencing from 2019,New Sale,1.0,1118000.0,-,721.0,Strata,01 to 05,1550.0,Feb-20
3,4,PARC CLEMATIS,JALAN LEMPENG,Apartment,5,OCR,99 yrs lease commencing from 2019,New Sale,1.0,1694000.0,-,1076.0,Strata,01 to 05,1574.0,Feb-20
4,5,KENT RIDGE HILL RESIDENCES,SOUTH BUONA VISTA ROAD,Apartment,5,RCR,99 yrs lease commencing from 2018,New Sale,1.0,1418000.0,-,775.0,Strata,01 to 05,1830.0,Feb-20


In [ ]:
# Running testing for landed since its sample size is smaller
